In [1]:
from module import Preprocess, Train

Call tf.config.experimental.set_memory_growth(GPU0, True)


# Global Variable & Functions

In [2]:
def print_console(message):
    print('-' * 10 + message + '-' * 10)

# Preprocessing

##### Loading Preprocess Module

In [3]:
preprocess = Preprocess(DATASET='ms_coco', print_console=print_console)

##### Loading Dataset
- It loads ```DATASET_SIZE``` size of pictures and captions.
- It returns:
    - ```train_ids```: Contains IDs of all the images
    - ```train_captions```: Contains captions of all the images
        - It contains ***start*** and ***end*** captions
    - ```img_name_vector```: Contains directory of all the images
    - ```corpus```: Contains reference sentences of all the images

In [4]:
train_ids, train_captions, img_name_vector, corpus = preprocess.load_dataset()

----------Loading MS-COCO Dataset----------
----------Finished Loading Dataset----------


##### Extracting Features using Inceptionv3 and YOLOv4
- It extracts using Inceptionv3 ann YOLOv4 models and concatenates the results, and saving the file in .npy extension.

In [ ]:
preprocess.feature_extraction(img_name_vector=img_name_vector)

##### Tokenizing
- It tokenizes the ```train_captions``` and also pads each vector to the max length of the captions
- It returns:
    - ```tokenizer```: Keras preprocessing module that has fit ```train_captions```
    - ```cap_vector```: It pads each vector to the max length of the caption
    - ```max_length```: It contains the max length of caption

In [5]:
tokenizer, cap_vector, max_length = preprocess.tokenize(train_captions=train_captions)

----------Tokenizing Captions----------
----------Finished Tokenizing Captions----------


##### Splitting Dataset into Train and Validation Set
- It splits the ```train_ids```, ```img_name_vector``` and ```cap_vector``` into train, validation set with proportion of ```TEST_SET_PROPORTION```
- It returns:
    - ```img_name_train```: Training set that contains full directory of images
    - ```cap_train```: Training set that contains tokenized padded captions of images

In [6]:
_, _, img_name_train, _, cap_train, _ = preprocess.train_test_split(train_ids=train_ids,
                                                                    img_name_vector=img_name_vector,
                                                                    cap_vector=cap_vector)

----------Splitting Dataset into Train and Validation Sets----------
----------Splitting Dataset into Train and Validation Sets----------


# Training

##### Loading Training Module

In [7]:
trainer = Train(print_console=print_console, NUM_STEPS=len(img_name_train))

##### Creating a Dataset for training
- It creates a dataset using TensorFlow module for faster training

In [8]:
dataset = trainer.make_dataset(img_name_train=img_name_train, cap_train=cap_train)

----------Making Dataset for Training----------
----------Finished Making Dataset for Training----------


##### Training
- It trains for ```EPOCHS``` number of epochs if ```RESET``` is true, else loads previously saved weights
    - It saves the weights as ```model_weights``` file

In [9]:
trainer.train(tokenizer=tokenizer, dataset=dataset)

----------Start Training----------


InvalidArgumentError: 1-th value returned by pyfunc_0 is int32, but expects int64
	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

# Evaluating